# This notebook goes through building transit network from gtfs to network standard

1. extract represetative trips
2. snap stops to roadway nodes
3. route bus on roadway via osmnx routing
4. route bus on roadway via shst routing
5. build non-bus/rail links and nodes
6. complete network node list that each transit path traverses
7. frquence based stop time
8. write out to transit network standard
9. write out quick QA/QC transit route true shape
10. write out network standard with rail nodes and links

In [1]:
import partridge as ptg
import peartree as pt
#%matplotlib inline
import requests
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import networkx as nx
from shapely import wkt
from scipy.spatial import cKDTree
import osmnx as ox
from dbfread import DBF
from osgeo import ogr
import glob
import time
import json

In [2]:
cd = "../gtfs_transit/"
muni_url = "https://transitfeeds.com/p/sfmta/60/20160125/download"

#osm drive file
link_file = "../tests/networkstandard/step1_roadway/sf_link.json"
with open(link_file) as f:
    link_json = json.load(f)
link_df = pd.DataFrame(link_json)

#osm drive file
node_file = "../tests/networkstandard/step1_roadway/sf_node.geojson"
node_gdf = gpd.read_file(node_file)

shape_gdf = gpd.read_file("../tests/networkstandard/step1_roadway/sf_shape.geojson")

In [431]:
link_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74352 entries, 0 to 74351
Data columns (total 39 columns):
LANES                 38653 non-null float64
access                74352 non-null object
area                  74352 non-null object
bike_access           74352 non-null int64
bridge                74352 non-null object
drive_access          74352 non-null int64
est_width             74352 non-null object
forward               41385 non-null float64
fromIntersectionId    74352 non-null object
highway               74352 non-null object
id                    74352 non-null object
junction              74352 non-null object
key                   74352 non-null object
landuse               74352 non-null object
lanes                 74352 non-null object
length                74352 non-null float64
link                  74352 non-null object
maxspeed              74352 non-null object
name                  74352 non-null object
nodeIds               74352 non-null object
oneWay    

In [15]:
print(node_gdf.crs)
print(node_gdf.columns)

{'init': 'epsg:4326'}
Index(['osm_node_id', 'shst_node_id', 'drive_access', 'walk_access',
       'bike_access', 'geometry'],
      dtype='object')


In [360]:
node_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 27700 entries, 0 to 27699
Data columns (total 6 columns):
osm_node_id     27700 non-null int64
shst_node_id    27700 non-null object
drive_access    27700 non-null int64
walk_access     27700 non-null int64
bike_access     27700 non-null int64
geometry        27700 non-null object
dtypes: int64(4), object(2)
memory usage: 1.3+ MB


In [3]:
# gfts shape shst match reference Id
muni_shst_df = gpd.read_file("../shst_node_js_match/muni.transit.out.matched.geojson")

In [4]:
drive_node_gdf = node_gdf[node_gdf.drive_access == 1].copy()
drive_link_df = link_df[link_df.drive_access == 1].copy()

In [5]:
def ox_graph(nodes_df, links_df):
    """
        create an osmnx-flavored network graph
        osmnx doesn't like values that are arrays, so remove the variables
        that have arrays.  osmnx also requires that certain variables
        be filled in, so do that too.
        Parameters
        ----------
        nodes_df : GeoDataFrame
        link_df : GeoDataFrame
        Returns
        -------
        networkx multidigraph
    """
    try:
        graph_nodes = nodes_df.drop(
                ["inboundReferenceId", "outboundReferenceId"], axis=1
            )
    except:
        graph_nodes = nodes_df.copy()

    graph_nodes.gdf_name = "network_nodes"
    graph_nodes['id'] = graph_nodes['shst_node_id']

    graph_links = links_df.copy()
    graph_links['id'] = graph_links['shstReferenceId']
    graph_links['key'] = graph_links['shstReferenceId']

    G = ox.gdfs_to_graph(graph_nodes, graph_links)

    return G

In [8]:
# build network routing file for osmnx routing

G_drive_sf = ox_graph(drive_node_gdf,
                  drive_link_df)

In [9]:
nx.shortest_path(G_drive_sf, 293741891, 65284950, weight = "length")

[293741891,
 65290257,
 911547143,
 3593679267,
 423778249,
 65290252,
 5435466368,
 65290251,
 65290249,
 65290238,
 65290236,
 5435466213,
 5435466205,
 65290232,
 5435466333,
 5435466219,
 65290229,
 65290227,
 5435466158,
 65290225,
 5435466163,
 65281097,
 4911322443,
 5437055071,
 65284950]

In [10]:
def get_representative_feed_from_gtfs(work_dir, in_url, fetch = False):
    
    print('getting representative feed...')
    
    if fetch == True:
        #read and save zip from url
        resp = urlopen(in_url)
        zipfile = ZipFile(BytesIO(resp.read()))
    
    if fetch == True:
        zipfile.extractall(work_dir + "muni")
    
    file_loc = work_dir + "muni"
    
    # get feed for the busiest day
    feed = pt.get_representative_feed(file_loc)
    
    return feed

In [11]:
feed = get_representative_feed_from_gtfs(cd, muni_url, True)

getting representative feed...


In [12]:
# pick representatives for each route by direction, with most number of trip 
def get_representative_trip_for_route(feed):
    
    """
    get the representative trips for each route, by direction, tod
    
    """
    
    print('getting representative trip...')
    
    # get the first stop of each trip, process time
    stop_times_df = feed.stop_times.copy()
    stop_times_df['arrival_h'] = pd.to_datetime(stop_times_df['arrival_time'], unit = 's').dt.hour
    stop_times_df['arrival_m'] = pd.to_datetime(stop_times_df['arrival_time'], unit = 's').dt.minute
    stop_times_df['departure_h'] = pd.to_datetime(stop_times_df['departure_time'], unit = 's').dt.hour
    stop_times_df['departure_m'] = pd.to_datetime(stop_times_df['departure_time'], unit = 's').dt.minute
    first_stop_df = stop_times_df[stop_times_df['stop_sequence'] == 1]
    
    ## identify peak, offpeak trips, based on the arrival time of first stop
    trip_df = feed.trips.copy()
    trip_df = pd.merge(trip_df, first_stop_df,
                      how = 'left',
                      on = 'trip_id')
    ## peak: 6-9am, offpeak: 9am-3pm
    trip_df['tod'] = np.where((trip_df['arrival_h'] >= 6) & (trip_df['arrival_h'] < 9),
                             'peak',
                             np.where((trip_df['arrival_h'] >= 9) & (trip_df['arrival_h'] < 15),
                             'offpeak',
                             'other'))
  
    # get the most frequent trip for each route, by direction, by time of day
    ## trips share the same shape_id is considered being the same
    ## first get the trip count for each shape_id
    trip_freq_df = trip_df.groupby(['route_id', 'tod', 'direction_id', 'shape_id'])['trip_id'].count().\
                            to_frame().\
                            drop(index = 'other', level = 1).\
                            reset_index()
    
    ## then choose the most frequent shape_id for each route, frequency use the total number of trips
    def agg(x):
        m = x.shape_id.iloc[np.argmax(x.trip_id.values)]
        return pd.Series({'trip_num' : x.trip_id.sum(), 'shape_id' : m})
   
    trip_freq_df = trip_freq_df.reset_index().groupby(['route_id', 'tod', 'direction_id']).apply(agg)
    
    # retain the complete trip info of represent trip only
    trip_df = pd.merge(trip_df, trip_freq_df.reset_index(),
                      how = 'inner',
                      on = ['route_id', 'tod', 'direction_id', 'shape_id']).\
                drop_duplicates(['route_id', 'direction_id', 'tod'])
    
    return trip_df

In [13]:
trip_df = get_representative_trip_for_route(feed)

getting representative trip...


In [14]:
print(trip_df.info())
trip_df[trip_df.route_id == "1031"]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207 entries, 0 to 4506
Data columns (total 21 columns):
route_id               207 non-null object
service_id             207 non-null object
trip_id                207 non-null object
trip_headsign          207 non-null object
direction_id           207 non-null int64
block_id               207 non-null object
shape_id               207 non-null object
arrival_time           207 non-null float64
departure_time         207 non-null float64
stop_id                207 non-null object
stop_sequence          207 non-null float64
stop_headsign          207 non-null object
pickup_type            0 non-null float64
drop_off_type          207 non-null object
shape_dist_traveled    0 non-null float64
arrival_h              207 non-null float64
arrival_m              207 non-null float64
departure_h            207 non-null float64
departure_m            207 non-null float64
tod                    207 non-null object
trip_num               207 non

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,arrival_time,departure_time,stop_id,...,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,arrival_h,arrival_m,departure_h,departure_m,tod,trip_num
188,1031,1,6682885,Downtown,1,5106,133152,25200.0,25200.0,3927,...,,NaN,,NaN,7.0,0.0,7.0,0.0,peak,13
201,1031,1,6682964,Downtown,1,5117,133152,32700.0,32700.0,3927,...,,NaN,,NaN,9.0,5.0,9.0,5.0,offpeak,1


In [16]:
def snap_stop_to_node(feed, node_gdf):
    
    """
    map gtfs stops to roadway nodes
    
    Parameters:
    ------------
    feed
    drive nodes
    
    return
    ------------
    stops with drive nodes id
    """
    
    print('snapping gtfs stops to roadway node osmid...')
    
    node_non_c_gdf = node_gdf.copy()
    node_non_c_gdf = node_non_c_gdf.to_crs({'init' : 'epsg:26915'})
    node_non_c_gdf['X'] = node_non_c_gdf.geometry.map(lambda g:g.x)
    node_non_c_gdf['Y'] = node_non_c_gdf.geometry.map(lambda g:g.y)
    inventory_node_ref = node_non_c_gdf[['X', 'Y']].values
    tree = cKDTree(inventory_node_ref)
    
    stop_df = feed.stops.copy()
    stop_df['geometry'] = [Point(xy) for xy in zip(stop_df['stop_lon'], stop_df['stop_lat'])]
    stop_df = gpd.GeoDataFrame(stop_df)
    stop_df.crs = {'init' : 'epsg:4326'}
    stop_df = stop_df.to_crs({'init' : 'epsg:26915'})
    stop_df['X'] = stop_df['geometry'].apply(lambda p: p.x)
    stop_df['Y'] = stop_df['geometry'].apply(lambda p: p.y)
   
    for i in range(len(stop_df)):
        point = stop_df.iloc[i][['X', 'Y']].values
        dd, ii = tree.query(point, k = 1)
        add_snap_gdf = gpd.GeoDataFrame(node_non_c_gdf.iloc[ii]).transpose().reset_index(drop = True)
        add_snap_gdf['stop_id'] = stop_df.iloc[i]['stop_id']
        if i == 0:
            stop_to_node_gdf = add_snap_gdf.copy()
        else:
            stop_to_node_gdf = stop_to_node_gdf.append(add_snap_gdf, ignore_index=True, sort=False)
    
    stop_df.drop(['X','Y'], axis = 1, inplace = True)
    stop_to_node_gdf = pd.merge(stop_df, stop_to_node_gdf, how = 'left', on = 'stop_id')
    
    column_list = feed.stops.columns.values.tolist() + ['osm_node_id', 'shst_node_id']
    
    return stop_to_node_gdf[column_list]

In [17]:
stop_df = snap_stop_to_node(feed, drive_node_gdf)

snapping gtfs stops to roadway node osmid...


In [18]:
stop_df.info()
stop_df.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3519 entries, 0 to 3518
Data columns (total 9 columns):
stop_id         3519 non-null object
stop_name       3519 non-null object
stop_desc       3519 non-null object
stop_lat        3519 non-null float64
stop_lon        3519 non-null float64
zone_id         3519 non-null object
stop_url        3519 non-null object
osm_node_id     3519 non-null object
shst_node_id    3519 non-null object
dtypes: float64(2), object(7)
memory usage: 274.9+ KB


,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,osm_node_id,shst_node_id
0,390,19th Avenue & Holloway St,,37.721190,-122.475096,,,3527108528,5014094a822c8a55290ed109a43d3687
1,913,DUBLIN ST & LAGRANDE AVE,,37.719192,-122.425802,,,4808718391,ac144f0c0a9977cf30163bf1a5309cc9
2,3003,2nd St & Brannan St,,37.781827,-122.391945,,,65315466,730a5b5ffca71fa58473cf5df382acae


In [50]:
feed.stop_times[feed.stop_times.trip_id == "6681228"]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
10075,6681228,38400.0,38400.0,4277,1,,NaN,,NaN
10076,6681228,38450.0,38450.0,3555,2,,NaN,,NaN
10077,6681228,38480.0,38480.0,3548,3,,NaN,,NaN
10078,6681228,38515.0,38515.0,3546,4,,NaN,,NaN
10079,6681228,38565.0,38565.0,3844,5,,NaN,,NaN
10080,6681228,38596.0,38596.0,3842,6,,NaN,,NaN
10081,6681228,38657.0,38657.0,3840,7,,NaN,,NaN
10082,6681228,38722.0,38722.0,3838,8,,NaN,,NaN
10083,6681228,38783.0,38783.0,3836,9,,NaN,,NaN
10084,6681228,38845.0,38845.0,3834,10,,NaN,,NaN


In [164]:
def route_bus_link_osmnx(roadway_gdf, node_gdf, G, feed, trip, stop):
    
    """
    route bus with OSMNX routing
    
    Parameters
    ----------
    drive link
    drive node
    drive graph
    feed
    trip 
    stop
    
    return
    ----------
    dataframe of drive links bus trips traverses
    list of trips that could not be routed by OSMNX
    """
    
    trip_df = trip.copy()
    stop_df = stop.copy()
    stop_time_df = feed.stop_times.copy()
    
    chained_stop_df = stop_time_df[stop_time_df['trip_id'].isin(trip_df.trip_id.tolist())]
    chained_stop_to_node_df = pd.merge(chained_stop_df, 
                                       stop_df,
                                        how = 'left',
                                        on = 'stop_id')
    
    print('routing bus on roadway network with osmnx...')
    
    #osm_node_dict = dict(zip(node_gdf.osmid, node_gdf.N))
    
    trip_df = pd.merge(trip_df, feed.routes, how = 'left', on = 'route_id')
    bus_trip_df = trip_df[trip_df['route_type'] == 3]
    
    # to track trips that osmnx failed to route
    broken_shape_trip_list = []
    
    # output dataframe for osmnx success
    trip_link_shape_df = pd.DataFrame()
    
    # loop through for bus trips
    for trip_id in bus_trip_df.trip_id.unique():
        
        # get the stops on the trip
        trip_stop_df = chained_stop_to_node_df[chained_stop_to_node_df['trip_id'] == trip_id]

        try:
            print("routing" + trip_id)
            for s in range(len(trip_stop_df)-1):
                # from stop node OSM id
                closest_node_to_stop1 = int(trip_stop_df.osm_node_id.iloc[s])
                
                # to stop node OSM id
                closest_node_to_stop2 = int(trip_stop_df.osm_node_id.iloc[s+1])
                
                # osmnx routing btw from and to stops, return the list of nodes
                node_osmid_list = nx.shortest_path(G, closest_node_to_stop1, closest_node_to_stop2)
                
                # get the links
                if len(node_osmid_list) > 1:
                    osm_link_gdf = pd.DataFrame({'u' : node_osmid_list[:len(node_osmid_list)-1], 
                                            'v' : node_osmid_list[1:len(node_osmid_list)],
                                            'trip_id' : trip_id},
                                               )
                else:
                    continue
                
                trip_link_shape_df = trip_link_shape_df.append(osm_link_gdf, ignore_index = True, sort = False)
                
        

        except:
            broken_shape_trip_list = broken_shape_trip_list + [trip_id]
            print('  warning: cannot route bus: ' + trip_id)
            continue      
                
    trip_link_shape_df = pd.merge(trip_link_shape_df, trip_df[['trip_id', 'shape_id']], how = 'left', on = 'trip_id')

    trip_link_shape_df = pd.merge(trip_link_shape_df,
                                  drive_link_df[["u", "v", "wayId", "shstReferenceId", "shstGeometryId"]].\
                                      drop_duplicates(subset = ["u", "v"]),
                                  how = "left",
                                  on = ["u", "v"])
    
    
    return trip_link_shape_df, broken_shape_trip_list

In [152]:
stop_df[stop_df.stop_id.isin(["3042", "3071"])]

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,osm_node_id,shst_node_id
31,3042,Balboa St & 14th Ave,,37.776780,-122.472700,,,65349672,925fee006f9bc6713d7bd75ec1058c2e
58,3071,Balboa St & Park Presidio Blvd,,37.776788,-122.472411,,,65349672,925fee006f9bc6713d7bd75ec1058c2e


In [165]:
bus_osmnx_link_shape_df, bus_osmnx_broken_trip_list = route_bus_link_osmnx(drive_link_df, 
                                                                            drive_node_gdf, 
                                                                            G_drive_sf, 
                                                                            feed,
                                                                            trip_df, 
                                                                            stop_df)

routing bus on roadway network with osmnx...
routing6681228
routing6681348
routing6681070
routing6681038
routing6682885
routing6682964
routing6750403
routing6750363
routing6750633
routing6750512
routing6750546
routing6750542
routing6697413
routing6697395
routing6697454
routing6697451
routing6697866
routing6697854
routing6697786
routing6697783
routing6791146
routing6791088
routing6807612
routing6791286
routing6700945
routing6700931
routing6700975
routing6781967
routing6781927
routing6781748
routing6781830
routing6702628
routing6702620
routing6702699
routing6702697
routing6703466
routing6703443
routing6703539
routing6703514
routing6704443
routing6704417
routing6704559
routing6704528
routing6771815
routing6771732
routing6771828
routing6771745
routing6705657
routing6705642
routing6705784
routing6705783
routing6707276
routing6707454
routing6707041
routing6706965
routing6706925
routing6707189
routing6709610
routing6709598
routing6709688
routing6709676
routing6710466
routing6710501
routing671

In [166]:
bus_osmnx_link_shape_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26921 entries, 0 to 26920
Data columns (total 7 columns):
u                  26921 non-null int64
v                  26921 non-null int64
trip_id            26921 non-null object
shape_id           26921 non-null object
wayId              26921 non-null object
shstReferenceId    26921 non-null object
shstGeometryId     26921 non-null object
dtypes: int64(2), object(5)
memory usage: 1.6+ MB


In [23]:
# osmnx failed to route these trips: can be rail modes
print(bus_osmnx_broken_trip_list)

['6750403', '6750363', '6702628', '6702620', '6702699', '6702697', '6703466', '6703443', '6704443', '6704417', '6771815', '6771732', '6707276', '6707454', '6717535', '6717599', '6720615', '6720606', '6794573', '6794555', '6794610', '6794606', '7089974', '7090030', '7089823', '7089797', '7092184']


In [27]:
# shapes that were not successfully routed by OSMNX

trip_df[trip_df.trip_id.isin(bus_osmnx_broken_trip_list)].shape_id.unique()

array(['133186', '133271', '133272', '133274', '133277', '134212',
       '133309', '133380', '133397', '133407', '133408', '138954',
       '138943', '138973'], dtype=object)

In [177]:
def route_bus_link_shst(drive_link, gtfs_shst_id):
    
    """
    route bus with shst match result
    
    parameter
    ---------
    drive link
    gtfs shst match return
    
    return
    ---------
    dataframe of drive links bus traverses
    list of imcomplete bus shapes
    
    """
    
    drive_link_df = drive_link.copy()
    shape_shst_df = gtfs_shst_id.copy()

    shape_shst_df = pd.merge(shape_shst_df, 
                             drive_link_df[['shstReferenceId','wayId','u','v', "fromIntersectionId", "toIntersectionId"]],
                             how = 'left',
                             left_on = 'shstReferenceId',
                             right_on = 'shstReferenceId')
    
    shape_shst_df["u"] = shape_shst_df["u"].fillna(0).astype(np.int64)
    shape_shst_df["v"] = shape_shst_df["v"].fillna(0).astype(np.int64)
    
    """shape_shst_df.dropna(subset = ['u','v'], 
                         axis = 0, 
                         inplace = True)"""
    
    shape_shst_df = shape_shst_df.reset_index(drop=True)
    
    shape_shst_df['next_pp_shape_id'] = shape_shst_df['pp_shape_id'].\
                                            iloc[1:].\
                                            append(pd.Series(shape_shst_df['pp_shape_id'].iloc[-1])).\
                                            reset_index(drop=True)
    
    shape_shst_df['next_u'] = shape_shst_df['u'].\
                                iloc[1:].\
                                append(pd.Series(shape_shst_df['v'].iloc[-1])).\
                                reset_index(drop=True)
    
    incomplete_shape_list = shape_shst_df[\
                                   (shape_shst_df.pp_shape_id==shape_shst_df.next_pp_shape_id)\
                                   &(shape_shst_df.v!=shape_shst_df.next_u)\
                                  ].pp_shape_id.unique().\
                                    tolist()
    
    shape_shst_df = shape_shst_df[~shape_shst_df.pp_shape_id.isin(incomplete_shape_list)].copy()
    
    return shape_shst_df, incomplete_shape_list

In [178]:
bus_shst_link_shape_df, incomplete_shape_list = route_bus_link_shst(drive_link_df, muni_shst_df)

print(bus_shst_link_shape_df.shape)
print(bus_shst_link_shape_df.pp_shape_id.nunique())

(31715, 24)
313


In [28]:
print(incomplete_shape_list)

[133153, 133158, 133188, 133192, 133194, 133199, 133202, 133206, 133209, 133213, 133214, 133225, 133226, 133232, 133249, 133251, 133276, 133308, 133312, 133327, 133336, 133346, 133349, 133351, 133389, 133392, 133418, 133423, 133435, 133436, 133473, 133494, 133640, 133701, 135377, 137828, 137832, 137836, 137842, 137847, 138105, 138277, 138338, 138341, 138342, 138608, 138938, 138943, 138946, 138951, 138953, 138957, 138958, 138975, 138976, 138993, 138997, 139044, 139072, 139132, 139135, 139137, 139150, 139154, 139216, 139281, 139294]


In [29]:
# these buses has parts that are out side of SF county boundary, that's why they are labeled as incomplete shape
bus_shst_link_shape_df[bus_shst_link_shape_df.fromIntersectionId.isnull()].pp_shape_id.unique()

array([], dtype=int64)

In [179]:
bus_shst_link_shape_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 31715 entries, 0 to 41872
Data columns (total 24 columns):
shstReferenceId           31715 non-null object
shstGeometryId            31715 non-null object
shstFromIntersectionId    31715 non-null object
shstToIntersectionId      31715 non-null object
gisReferenceId            31715 non-null object
gisGeometryId             31715 non-null object
gisTotalSegments          31715 non-null int64
gisSegmentIndex           31715 non-null int64
gisFromIntersectionId     31715 non-null object
gisToIntersectionId       31715 non-null object
startSideOfStreet         31715 non-null object
endSideOfStreet           31715 non-null object
sideOfStreet              31715 non-null object
score                     31715 non-null float64
matchType                 31715 non-null object
pp_shape_id               31715 non-null int64
geometry                  31715 non-null object
wayId                     31715 non-null object
u                    

In [40]:
def bus_link(bus_link_osmnx, bus_link_shst, trip, incomplete_list):
    
    """
    combine bus links from OSMNX and SHST
    """
    
    bus_link_osmnx_df = bus_link_osmnx.copy()
    bus_link_shst_df = bus_link_shst.copy()
    
    trip_df = trip.copy()
    trip_df = pd.merge(trip_df, feed.routes[['route_id', 'route_type']], how = 'left', on = 'route_id')
    bus_trip_df = trip_df[trip_df.route_type == 3].copy()
    
    bus_link_shst_df.pp_shape_id = bus_link_shst_df.pp_shape_id.astype(str)
    
    shape_id_list = bus_trip_df.shape_id.unique().tolist()
    
    incomplete_list = [str(x) for x in incomplete_list]
    
    print("Targeting number of bus shape IDs: " + str(bus_trip_df.shape_id.nunique()))
    
    #trip_id, shape_id, u, v, link_id, omsid, shstrefere
    
    shst_shape_list = list(set([str(x) for x in bus_link_shst_df.pp_shape_id]))
    
    shapes_replace_with_shst_list = [x for x in shst_shape_list if x in shape_id_list]
    
    print("\n There are " + str(len(shapes_replace_with_shst_list)) + 
          " shapes that are from shst gtfs matching: \n \t" + 
          str(shapes_replace_with_shst_list))

    bus_link_osmnx_df = bus_link_osmnx_df[~bus_link_osmnx_df.shape_id.isin(shapes_replace_with_shst_list)].copy()
    
    osmnx_shape_list = bus_link_osmnx_df.shape_id.unique().tolist()
    
    print("\n There are " + str(len(osmnx_shape_list)) + 
          " shapes that are from OSMNX routing: \n \t" + 
          str(osmnx_shape_list))
    
    not_routed_list = [x for x in shape_id_list if x not in (shst_shape_list + osmnx_shape_list + incomplete_list)]
    
    print("\n There are " + str(len(not_routed_list)) + 
         " shapes that are not routed by either of the two methods: \n \t" + 
         str(not_routed_list))
    
    bus_link_shst_df = pd.merge(bus_link_shst_df,
                                bus_trip_df[['trip_id', 'shape_id']],
                                how = 'inner',
                                left_on = 'pp_shape_id',
                                right_on = 'shape_id')
    #bus_link_shst_df.drop(['pp_shape_id'], axis = 1, inplace = True)
    
    bus_link_df = pd.concat([bus_link_osmnx_df, bus_link_shst_df],
                            sort = False,
                           ignore_index = True)
    
    column_list = bus_link_osmnx.columns.values.tolist()
    
    return bus_link_df[column_list]

In [180]:
bus_link_df = bus_link(bus_osmnx_link_shape_df, bus_shst_link_shape_df, trip_df, incomplete_shape_list)

Targeting number of bus shape IDs: 102

 There are 34 shapes that are from shst gtfs matching: 
 	['139047', '133395', '133432', '133332', '139001', '138979', '133401', '133267', '133433', '133426', '139050', '139139', '138961', '133142', '133586', '138908', '133299', '139022', '133235', '139024', '133139', '133396', '133368', '133309', '139026', '138980', '133331', '133422', '133260', '133274', '133238', '138928', '133305', '133328']

 There are 67 shapes that are from OSMNX routing: 
 	['133152', '133186', '133190', '133193', '133236', '133237', '133247', '133251', '133254', '133256', '133272', '133276', '133277', '133281', '134212', '134215', '133284', '133289', '133300', '133324', '133333', '133335', '133338', '133341', '133354', '133355', '133364', '133366', '133373', '133380', '133379', '133381', '133382', '133384', '133390', '133394', '133397', '133398', '133402', '133403', '133405', '133407', '133408', '133423', '133425', '133435', '138910', '138919', '138924', '138931', '13893

In [181]:
bus_link_df.info()
bus_link_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27402 entries, 0 to 27401
Data columns (total 7 columns):
u                  27402 non-null int64
v                  27402 non-null int64
trip_id            27402 non-null object
shape_id           27402 non-null object
wayId              27402 non-null object
shstReferenceId    27402 non-null object
shstGeometryId     27402 non-null object
dtypes: int64(2), object(5)
memory usage: 1.5+ MB


,u,v,trip_id,shape_id,wayId,shstReferenceId,shstGeometryId
0,65336459,999546633,6682885,133152,254297751,6bcc81150482ea540ad30935a652ece5,8d7fba9062dcae6163659bc6cec98138
1,999546633,4044971115,6682885,133152,254297751,0629790246e27ddf1e5aa739ad390e0d,cc5b3ce737fffcd34af7ce215e139d5a
2,4044971115,4044971130,6682885,133152,470212712,f2dcc7fc9d575752bbe11ddb1c03c7fb,f6345495ebcd1227a44bc5b6431f180e


In [216]:
# create rail links
def non_bus_link(feed, trip, stop):
    
    """
    create rail links and nodes
    
    nodes are based on rail stops, links are true shape between nodes
    
    return
    ---------
    complete rail link path for each rail service
    complete rail node path for each rail service
    
    """
    
    print('generating rail links...')
    
    #get rail trips
    trip_df = trip.copy()
    trip_df = pd.merge(trip_df, feed.routes[['route_id', 'route_type']], how = 'left', on = 'route_id')
    rail_trip_df = trip_df[trip_df.route_type != 3].copy()
    
    stop_df = stop.copy()
    stop_time_df = feed.stop_times.copy()
    
    #get rail trips with stops
    chained_stop_to_node_df = pd.merge(stop_time_df, 
                                       stop_df,
                                       how = 'left',
                                       on = 'stop_id')
    
    rail_stop_time_df = chained_stop_to_node_df[
                                                chained_stop_to_node_df['trip_id']\
                                                .isin(rail_trip_df.trip_id.tolist())
                                               ]\
                                                .copy()
    
    #get gtfs rail shapes
    rail_shape_df = feed.shapes[
                                feed.shapes['shape_id'].isin(rail_trip_df.shape_id.tolist())
                                ].copy()
    
    #gtfs shape-trip correspondence
    shape_trip_dict = dict(zip(rail_trip_df.shape_id, rail_trip_df.trip_id))
    
    #for each rail shape
    for i in rail_shape_df.shape_id.unique():
        trip_id = shape_trip_dict[i]
        
        #get chained stop
        trip_stop_df = rail_stop_time_df[rail_stop_time_df.trip_id == trip_id].copy()
        
        # get gtfs shape nodes for the shape
        trip_shape_df = rail_shape_df[rail_shape_df.shape_id == i].copy()
        # initialize columns
        trip_shape_df['is_stop'] = np.int(0)
        trip_shape_df['stop_id'] = np.nan
        
        # for each rail stop, find the closest node in the shape, and those are the stops and breakpoints of new rail links
        # return is a gtfs node shape dataframe with two columns indicating if the node is a stop and the stop id
        shape_inventory = trip_shape_df[['shape_pt_lon', 'shape_pt_lat']].values
        tree = cKDTree(shape_inventory)
        for s in range(len(trip_stop_df)):
            point = trip_stop_df.iloc[s][['stop_lon', 'stop_lat']].values
            dd, ii = tree.query(point, k = 1)
            trip_shape_df.is_stop.iloc[ii] = 1
            trip_shape_df.stop_id.iloc[ii] = trip_stop_df.iloc[s]['stop_id']
        
        # appending the gtfs shape for each route shape id
        if i == rail_shape_df.shape_id.unique()[0]:
            shape_flag_df = trip_shape_df.copy()
        else:
            shape_flag_df = shape_flag_df.append(trip_shape_df, 
                                                 ignore_index = True, 
                                                 sort = False)
    
    # starting to build new rail links true shape
    linestring_df = pd.DataFrame(columns = ['shape_id', 'u', 'v', 'geometry', 'u_stop_id', 'v_stop_id'])

    # rail links are based on the gtfs shape, with nodes being the shapes that are identified as rail stops.
    for i in shape_flag_df.shape_id.unique():
        # get gtfs shape for shape id
        shape_route_df = shape_flag_df[shape_flag_df.shape_id == i]
        
        # get rail nodes based on the stop flags
        break_list = shape_route_df.index[shape_route_df.is_stop == 1].tolist()
        stop_id_list = shape_route_df[shape_route_df.is_stop == 1]['stop_id'].tolist()
        
        # use the gtfs shape between "stop" shapes to build the rail true shape
        for j in range(len(break_list)-1):
            lon_list = shape_flag_df.shape_pt_lon.iloc[break_list[j]:break_list[j+1]+1].tolist()
            lat_list = shape_flag_df.shape_pt_lat.iloc[break_list[j]:break_list[j+1]+1].tolist()
            linestring = LineString([Point(xy) for xy in zip(lon_list,lat_list)])
            linestring_df = linestring_df.append({'shape_id':i, 
                                                  'u':break_list[j], 
                                                  'v':break_list[j+1],
                                                  'u_stop_id':stop_id_list[j], 
                                                  'v_stop_id':stop_id_list[j+1],
                                                  'geometry' : linestring}, 
                                                 ignore_index = True, 
                                                 sort = False)
    
    # add rail travel time between stops
    stop_time_df = pd.merge(
                            stop_time_df, 
                            rail_trip_df[['trip_id', 'shape_id']], 
                            how = 'left', 
                            on = 'trip_id')
    
    unique_stop_time_df = stop_time_df[
                                        stop_time_df.shape_id.notnull()
                                    ].groupby(['trip_id', 'shape_id'])\
                                    .count().reset_index()\
                                    .drop_duplicates(subset = ['shape_id']).copy()
    
    stop_time_df = stop_time_df[stop_time_df.trip_id.isin(unique_stop_time_df.trip_id.tolist())].copy()

    
    linestring_df = pd.merge(linestring_df, 
                             stop_time_df[['shape_id', 'stop_id' , 'departure_time']].rename(
                                 columns = {"stop_id" : "u_stop_id"}),
                            how = 'left',
                            on = ['shape_id', 'u_stop_id'])
    
    linestring_df = pd.merge(linestring_df, 
                             stop_time_df[['shape_id', 'stop_id', 'arrival_time']].rename(
                                 columns = {"stop_id" : "v_stop_id"}),
                            how = 'left',
                            on = ['shape_id', 'v_stop_id'])
    
    # travel time in minutes
    linestring_df['rail_traveltime'] = (linestring_df['arrival_time'] - linestring_df['departure_time'])/60
    
    rail_node_df = shape_flag_df[shape_flag_df.is_stop == 1].rename_axis('node_id').reset_index()

    
    return linestring_df, rail_node_df

In [244]:
%%time
rail_path_link_gdf, rail_path_node_df = non_bus_link(feed, trip_df, stop_df)

generating rail links...


C:\Users\wangs1\.conda\envs\MetC_new\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Wall time: 10min 46s


In [245]:
print(rail_path_node_df.columns)
print(rail_path_link_gdf.columns)

print(rail_path_node_df.shape)
print(rail_path_link_gdf.shape)

Index(['node_id', 'shape_id', 'shape_pt_lon', 'shape_pt_lat',
       'shape_pt_sequence', 'shape_dist_traveled', 'is_stop', 'stop_id'],
      dtype='object')
Index(['shape_id', 'u', 'v', 'geometry', 'u_stop_id', 'v_stop_id',
       'departure_time', 'arrival_time', 'rail_traveltime'],
      dtype='object')
(246, 8)
(237, 9)


In [219]:
rail_path_link_gdf

,shape_id,u,v,geometry,u_stop_id,v_stop_id,departure_time,arrival_time,rail_traveltime
0,133410,0,2,"LINESTRING (-122.407648 37.784394, -122.407853...",6063,6068,50280.0,50401.0,2.016667
1,133410,2,3,"LINESTRING (-122.408036 37.786402, -122.408231...",6068,6058,50401.0,50461.0,1.000000
2,133410,3,4,"LINESTRING (-122.408231 37.787357, -122.408402...",6058,6072,50461.0,50519.0,0.966667
3,133410,4,5,"LINESTRING (-122.408402 37.78828499999999, -12...",6072,6075,50519.0,50580.0,1.016667
4,133410,5,7,"LINESTRING (-122.408596 37.789222, -122.408734...",6075,6047,50580.0,50638.0,0.966667
5,133410,7,9,"LINESTRING (-122.408791 37.79015, -122.408928 ...",6047,6069,50638.0,50691.0,0.883333
6,133410,9,10,"LINESTRING (-122.408985 37.791087, -122.40918 ...",6069,6049,50691.0,50760.0,1.150000
7,133410,10,11,"LINESTRING (-122.40918 37.792032, -122.409363 ...",6049,6073,50760.0,50781.0,0.350000
8,133410,11,12,"LINESTRING (-122.409363 37.792969, -122.409546...",6073,6051,50781.0,50813.0,0.533333
9,133410,12,14,"LINESTRING (-122.409546 37.793772, -122.409546...",6051,6077,50813.0,50845.0,0.533333


In [214]:
rail_path_node_df

,node_id,shape_id,shape_pt_lon,shape_pt_lat,shape_pt_sequence,shape_dist_traveled,is_stop,stop_id
0,0,133410,-122.407648,37.784394,1,0,1,6063
1,2,133410,-122.408036,37.786402,3,227,1,6068
2,3,133410,-122.408231,37.787357,4,335,1,6058
3,4,133410,-122.408402,37.788285,5,439,1,6072
4,5,133410,-122.408596,37.789222,6,545,1,6075
5,7,133410,-122.408791,37.790150,8,650,1,6047
6,9,133410,-122.408985,37.791087,10,755,1,6069
7,10,133410,-122.409180,37.792032,11,862,1,6049
8,11,133410,-122.409363,37.792969,12,968,1,6073
9,12,133410,-122.409546,37.793772,13,1059,1,6051


In [91]:
unique_rail_node_gdf

,node_id,shape_id,shape_pt_lon,shape_pt_lat,shape_pt_sequence,shape_dist_traveled,is_stop,stop_id,osm_node_id,geometry,transit_access
0,0,133410,-122.407648,37.784394,1,0,1,6063,1,POINT (-122.407648 37.784394),1
1,2,133410,-122.408036,37.786402,3,227,1,6068,2,POINT (-122.408036 37.786402),1
2,3,133410,-122.408231,37.787357,4,335,1,6058,3,POINT (-122.408231 37.787357),1
3,4,133410,-122.408402,37.788285,5,439,1,6072,4,POINT (-122.408402 37.78828499999999),1
4,5,133410,-122.408596,37.789222,6,545,1,6075,5,POINT (-122.408596 37.789222),1
5,7,133410,-122.408791,37.790150,8,650,1,6047,6,POINT (-122.408791 37.79015),1
6,9,133410,-122.408985,37.791087,10,755,1,6069,7,POINT (-122.408985 37.791087),1
7,10,133410,-122.409180,37.792032,11,862,1,6049,8,POINT (-122.40918 37.792032),1
8,11,133410,-122.409363,37.792969,12,968,1,6073,9,POINT (-122.409363 37.792969),1
9,12,133410,-122.409546,37.793772,13,1059,1,6051,10,POINT (-122.409546 37.793772),1


In [445]:
def combine_bus_and_rail_shape(rail_path_link, rail_path_node, link, node, shape):
    
    """
    add only unique rail links and nodes to roadway standard
    
    parameter
    -----------
    complete rail link path
    complete rail node path
    all roadway links
    all roadway nodes
    all roadway shapes
    
    return
    -----------
    all roadway and rail links
    all roadway and rail nodes
    all roadway and rail shapes
    unique rail links
    unique rail nodes
    complete rail link path with updated link ID
    
    """
    
    print('indexing rail links and nodes...')
    
    node_gdf = node.copy()
    link_df = link.copy()
    shape_gdf = shape.copy()
    
    # add unique rail nodes to roadway node dataframe
    rail_path_node_gdf = rail_path_node.copy()
    
    unique_rail_node_df = rail_path_node_gdf.drop_duplicates(['shape_pt_lat', 'shape_pt_lon']).copy()
    
    # fake osm node id for rail nodes
    # save 10000 nodes for TAZ centroid
    taz_high = 10000
    
    unique_rail_node_df['osm_node_id'] = range(1 + taz_high, 1+ taz_high + len(unique_rail_node_df))
    
    rail_path_node_gdf = pd.merge(rail_path_node_gdf, 
                            unique_rail_node_df[['shape_pt_lat', 'shape_pt_lon', 'osm_node_id']], 
                            how = 'left', 
                            on = ['shape_pt_lat', 'shape_pt_lon'])
    
    # get unique rail nodes
    unique_rail_node_df['geometry'] = [Point(xy) for xy in zip(unique_rail_node_df.shape_pt_lon, 
                                                               unique_rail_node_df.shape_pt_lat)]
    
    unique_rail_node_df = gpd.GeoDataFrame(unique_rail_node_df)
    unique_rail_node_df.crs = {'init' : 'epsg:4326'}
    unique_rail_node_df = unique_rail_node_df.to_crs(node_gdf.crs)
    
    unique_rail_node_df['transit_access'] = int(1)
    unique_rail_node_df["walk_access"] = int(1)
    
    # combine rail nodes and roadway nodes
    node_gdf["transit_access"] = int(0)
    
    rail_node_columns = ["osm_node_id", "geometry", "transit_access", "walk_access"]
    
    roadway_and_rail_node_gdf = node_gdf.append(unique_rail_node_df[rail_node_columns],
                                                ignore_index = True, 
                                                sort = False)
    
    
    rail_node_osmid_dict = dict(zip(rail_path_node_gdf.node_id, rail_path_node_gdf.osm_node_id))
    
    rail_path_link_df = rail_path_link.copy()
    
    rail_path_link_df['u'] = rail_path_link_df.u.map(rail_node_osmid_dict)
    rail_path_link_df['v'] = rail_path_link_df.v.map(rail_node_osmid_dict)
    
    rail_path_link_df = gpd.GeoDataFrame(rail_path_link_df)
    rail_path_link_df.crs = {'init' : 'epsg:4326'}
    
    # get unique rail links
    unique_rail_link_gdf = rail_path_link_df.drop_duplicates(['u', 'v']).copy()
    
    # fake rail link osmid
    unique_rail_link_gdf['wayId'] = range(1, 1 + len(unique_rail_link_gdf))
    # fake rail link shst geom id
    unique_rail_link_gdf['shstGeometryId'] = unique_rail_link_gdf.wayId.apply(lambda x:'rail'+str(x))
    unique_rail_link_gdf['id'] = unique_rail_link_gdf['shstGeometryId']

    unique_rail_link_gdf['transit_access'] = int(1)
    
    rail_path_link_df = pd.merge(rail_path_link_df,
                                unique_rail_link_gdf[["u", "v", "shstGeometryId"]],
                                how = "left",
                                on = ["u", "v"])
    
    rail_link_columns = ['u', 'v', 'wayId', "shstGeometryId", "rail_traveltime","transit_access", "id"]
    rail_shape_columns = ["id", "geometry"]
    
    # combine rail and roadway links
    roadway_and_rail_link_df = link_df.append(unique_rail_link_gdf[rail_link_columns], 
                                              ignore_index = True, 
                                              sort = False)
    
    # combine rail and roadway shapes
    roadway_and_rail_shape_gdf = shape_gdf.append(unique_rail_link_gdf[rail_shape_columns],
                                                 ignore_index = True,
                                                 sort = False)
    
    """rail_path_link_df = pd.merge(rail_path_link_df[['shape_id', 'geometry', 'u_stop_id', 'v_stop_id']],
                            unique_rail_shape_gdf.drop(['geometry', 'shape_id'], axis = 1),
                            how = 'left',
                            on = ['u_stop_id', 'v_stop_id'])"""
    
    rail_path_link_df = rail_path_link_df.to_crs({'init' : 'epsg:4326'})
        
    return roadway_and_rail_link_df, roadway_and_rail_node_gdf, roadway_and_rail_shape_gdf, \
                unique_rail_link_gdf, unique_rail_node_df, \
                rail_path_link_df

In [446]:
roadway_and_rail_link_df, roadway_and_rail_node_gdf, roadway_and_rail_shape_gdf, unique_rail_link_gdf, unique_rail_node_gdf, \
                                            rail_link_gdf = combine_bus_and_rail_shape(
                                                                                      rail_path_link_gdf, 
                                                                                      rail_path_node_df,
                                                                                      link_df, 
                                                                                      node_gdf,
                                                                                      shape_gdf)

indexing rail links and nodes...


In [447]:
roadway_and_rail_shape_gdf.id.nunique()

41611

In [448]:
roadway_and_rail_link_df.id.nunique()

41611

In [399]:
unique_rail_node_gdf.shape

(152, 12)

In [400]:
unique_rail_link_gdf.shape

(226, 13)

In [401]:
print(link_df.shape)
print(node_gdf.shape)
print(shape_gdf.shape)

(74352, 39)
(27700, 6)
(41385, 6)


In [402]:
print(roadway_and_rail_node_gdf.shape)
print(roadway_and_rail_link_df.shape)
print(roadway_and_rail_shape_gdf.shape)

(27852, 7)
(74578, 41)
(41611, 6)


In [71]:
def create_freq_table(trip_df):
    
    """
    create frequency table for network standard
    """
    
    print('creating frequency reference...')
    
    freq_df = trip_df[['trip_id', 'tod', 'direction_id', 'trip_num']].copy()
    freq_df['headway_secs'] = np.where(freq_df.tod == 'peak', (3*60*60/freq_df.trip_num).astype(int),
                      (6*60*60/freq_df.trip_num).astype(int))
    freq_enum_list = {'start_time' : {'peak' : '06:00:00', 'offpeak' : '09:00:00'},
                      'end_time' : {'peak' : '09:00:00', 'offpeak' : '15:00:00'}}
    
    freq_df['start_time'] = freq_df.tod.map(freq_enum_list.get("start_time"))
    freq_df['end_time'] = freq_df.tod.map(freq_enum_list.get("end_time"))
    
    return freq_df

In [72]:
freq_df = create_freq_table(trip_df)

creating frequency reference...


In [403]:
# create new shape with complete node list the route passes
def create_new_node_shape(node, bus_link, rail_link = pd.DataFrame(columns = ["u", "v", "shape_id"])):
    
    """
    create complete node lists each transit traverses to replace the gtfs shape.txt
    """
    
    shape_link_df = pd.concat([bus_link[['u', 'v', 'shape_id']]
                                , rail_link[['u', 'v', 'shape_id']]],
                               sort = False,
                               ignore_index = True)

    shape_link_df.drop_duplicates(subset = ["shape_id", "u", "v"], inplace = True)

    shape_point_df = gpd.GeoDataFrame()
    
    for shape_id in shape_link_df.shape_id.unique():
        shape_df = shape_link_df[shape_link_df.shape_id == shape_id]
        point_df = pd.DataFrame(data = {"shape_id" : shape_id,
                                         "shape_osm_node_id" : shape_df.u.tolist() + [shape_df.v.iloc[-1]],
                                       "shape_pt_sequence" : range(1, 1+len(shape_df)+1)})
   
        shape_point_df = pd.concat([shape_point_df,
                                   point_df],
                                  sort = False,
                                  ignore_index = True)

    shape_point_df = pd.merge(shape_point_df,
                             node[["osm_node_id", "shst_node_id", "geometry"]],
                             how = "left",
                             left_on = "shape_osm_node_id",
                             right_on = "osm_node_id")
    
    shape_point_df.crs = {'init' : 'epsg:4326'}
    #shape_point_df = shape_point_df.to_crs(epsg = 4326)
    
    shape_point_df["shape_pt_lat"] = shape_point_df.geometry.map(lambda g:g.y)
    shape_point_df["shape_pt_lon"] = shape_point_df.geometry.map(lambda g:g.x)
    
    shape_point_df.rename(columns = {"shst_node_id":"shape_shst_node_id"}, inplace = True)
        
    return shape_point_df[["shape_id", "shape_pt_sequence", "shape_osm_node_id", "shape_shst_node_id"]]

In [404]:
shape_point_df = create_new_node_shape(roadway_and_rail_node_gdf, bus_link_df, rail_link_gdf)

C:\Users\wangs1\.conda\envs\MetC_new\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [405]:
shape_point_df

,shape_id,shape_pt_sequence,shape_osm_node_id,shape_shst_node_id
0,133152,1,65336459,1cf517d6a87edce47bb115c7c3ce61a9
1,133152,2,999546633,28ca7c7929a26cc483154a9b7bb50094
2,133152,3,4044971115,4edfe3c72a7fee9c4b339cdceaee1042
3,133152,4,4044971130,6c4d8e3c61f81406db91256f02c427ec
4,133152,5,65331040,bb6fd5412ee517218d4580862843ba8e
5,133152,6,65322190,8c33fe8a81edc7c1214d58e1cf2716f7
6,133152,7,65309742,9a1b0d33b5f3f50471de0e07df3006f6
7,133152,8,65366859,9546d988da125d86f56cbd7f4f0e0e17
8,133152,9,65349699,55f21767ce943793b198b804da35cff5
9,133152,10,4021327368,498bff9636299036e98a0ce469dc61f5


In [406]:
shape_point_df.shape_id.nunique()

110

In [408]:
def write_out_transit_standard(trip, stop, shape_point, freq, feed, url, rail_node = False):
    
    shape_point_df = shape_point.copy()
    trip_df = trip.copy()
    
    trip_df = trip_df[trip_df.shape_id.isin(shape_point_df.shape_id.unique().tolist())]
    
    final_trip_list = trip_df.trip_id.unique().tolist()
    
    freq_df = freq.copy()
    freq_df = freq_df[freq_df.trip_id.isin(final_trip_list)]
    
    stop_df = stop.copy()
    
    if type(rail_node) != bool:
        rail_node_df = rail_node.copy()
        rail_node_dict = dict(zip(rail_node_df.stop_id, rail_node_df.osm_node_id))
        
        stop_df['osm_node_id'] = stop_df.apply(lambda x: rail_node_dict[x.stop_id] 
                                               if x.stop_id in rail_node_df.stop_id.tolist() 
                                               else x.osm_node_id,
                                                axis = 1)
        stop_df['shst_node_id'] = stop_df.apply(lambda x: '' 
                                                if x.stop_id in rail_node_df.stop_id.tolist() 
                                                else x.shst_node_id,
                                                axis = 1)
    

    stop_times_df = feed.stop_times.copy()
    stop_times_df = stop_times_df[stop_times_df.trip_id.isin(final_trip_list)]
    
    # update time to relative time for frequency based transit system
    stop_times_df['first_arrival'] = stop_times_df.groupby(['trip_id'])['arrival_time'].transform(min)
    stop_times_df['arrival_time'] = stop_times_df['arrival_time'] - stop_times_df['first_arrival']
    stop_times_df['departure_time'] = stop_times_df['departure_time'] - stop_times_df['first_arrival']
    stop_times_df['arrival_time'] = stop_times_df['arrival_time'].apply(lambda x : time.strftime('%H:%M:%S', 
                                                                                                 time.gmtime(x)))
    stop_times_df['departure_time'] = stop_times_df['departure_time'].apply(lambda x : time.strftime('%H:%M:%S', 
                                                                                                     time.gmtime(x)))
    stop_times_df.drop(['first_arrival'], axis = 1, inplace = True)
    
    route_df = feed.routes.copy()
    route_df = route_df[route_df.route_id.isin(trip_df.route_id.tolist())]
    
    route_df.to_csv('../tests/networkstandard/step2_transit/routes.txt', 
                    index = False, 
                    sep = ',')
    shape_point_df.to_csv('../tests/networkstandard/step2_transit/shapes.txt', 
                          index = False, 
                          sep = ',')
    trip_df[feed.trips.columns.values].to_csv('../tests/networkstandard/step2_transit/trips.txt', 
                                              index = False, 
                                              sep = ',')
    freq_df[['trip_id', 'headway_secs', 'start_time', 'end_time']].to_csv('../tests/networkstandard/step2_transit/frequencies.txt', 
                                                index = False, 
                                                sep = ',')
    stop_df.to_csv('../tests/networkstandard/step2_transit/stops.txt', 
                   index = False, 
                   sep = ',')
    stop_times_df.to_csv('../tests/networkstandard/step2_transit/stop_times.txt', 
                         index = False, 
                         sep = ',')


In [409]:
write_out_transit_standard(trip_df, 
                           stop_df, 
                           shape_point_df, 
                           freq_df, 
                           feed,
                           muni_url, 
                           unique_rail_node_gdf)

In [449]:
def create_transit_access_link(all_link, all_node, all_shape):
    
    """
    create rail walk access/egress links
    """
    
    tran_node_df = all_node[all_node.transit_access == 1].copy()
    walk_node_df = all_node[(all_node.walk_access == 1) & (all_node.transit_access == 0)].copy().reset_index(drop = True)
    
    walk_node_df = walk_node_df.to_crs({'init' : 'epsg:26915'})
    walk_node_df['X'] = walk_node_df.geometry.map(lambda g:g.x)
    walk_node_df['Y'] = walk_node_df.geometry.map(lambda g:g.y)
    inventory_node_ref = walk_node_df[['X', 'Y']].values
    tree = cKDTree(inventory_node_ref)
    
    tran_node_df = tran_node_df.to_crs({'init' : 'epsg:26915'})
    tran_node_df['X'] = tran_node_df.geometry.map(lambda g:g.x)
    tran_node_df['Y'] = tran_node_df.geometry.map(lambda g:g.y)
    
    for i in range(len(tran_node_df)):
        point = tran_node_df.iloc[i][['X', 'Y']].values
        dd, ii = tree.query(point, k = 1)
        add_node_gdf = gpd.GeoDataFrame(walk_node_df.iloc[ii]).transpose().reset_index(drop = True)
        add_node_gdf['tran_node'] = tran_node_df.iloc[i].osm_node_id
        add_node_gdf['geometry_tran'] = tran_node_df.iloc[i].geometry
        
        if i == 0:
            rail_access_gdf = add_node_gdf.copy()
        else:
            rail_access_gdf = rail_access_gdf.append(add_node_gdf, ignore_index=True, sort=False)
    
    rail_access_gdf.rename(columns = {'geometry' : "geometry_walk"}, inplace = True)

    
    rail_access_gdf['geometry'] = [LineString(xy) for xy in zip(rail_access_gdf['geometry_walk'], 
                                                                rail_access_gdf['geometry_tran'])]
    
    # largest transit link id
    rail_link_osmid_high = all_link[all_link.transit_access == 1].wayId.max()
    
    # fake rail link osmid
    rail_access_gdf['wayId'] = range(1 + rail_link_osmid_high, 
                                     1 + rail_link_osmid_high + len(rail_access_gdf))
    # fake rail link shst geom id
    rail_access_gdf['shstGeometryId'] = rail_access_gdf.wayId.apply(lambda x:'walktorail'+str(x))
    rail_access_gdf['id'] = rail_access_gdf['shstGeometryId']
    
    rail_access_gdf["fromIntersectionId"] = rail_access_gdf.shst_node_id

    rail_access_gdf_copy = rail_access_gdf.copy()
    rail_access_gdf.rename(columns = {'osm_node_id' : 'u', 'tran_node' : 'v'}, inplace = True)
    
    rail_access_gdf_copy.rename(columns = {'tran_node' : 'u', 'osm_node_id' : 'v'}, inplace = True)
    
    rail_access_gdf = pd.concat(
                            [rail_access_gdf[['u', 'v', 'geometry', "wayId", 'shstGeometryId', "id", "fromIntersectionId"]],
                            rail_access_gdf_copy[['u', 'v', 'geometry',"wayId", 'shstGeometryId', "id", "fromIntersectionId"]]],
                               ignore_index = True,
                               sort = False)
    
    rail_access_gdf = gpd.GeoDataFrame(rail_access_gdf)
    rail_access_gdf.crs = {'init' : 'epsg:26915'}
    rail_access_gdf = rail_access_gdf.to_crs(all_node.crs)
    
    rail_access_gdf['walk_access'] = 1
    
    rail_access_link_columns = ["u", "v", "wayId", "shstGeometryId", "walk_access", "id"]
    rail_access_shape_columns = ["id", "fromIntersectionId", "geometry"]
    
    all_link_df = all_link.copy()
    all_shape_gdf = all_shape.copy()
    
        
    all_shape_gdf = pd.concat([
                                all_shape_gdf,
                                rail_access_gdf[rail_access_shape_columns].drop_duplicates(
                                                                        subset = ["id"])
                              ],
                             sort = False,
                             ignore_index= True)

    
    all_link_df = pd.concat([all_link_df, 
                             rail_access_gdf[rail_access_link_columns]], 
                            ignore_index = True, 
                            sort = False)
    
    all_link_gdf = pd.merge(all_link_df,
                           all_shape_gdf,
                           how = "left",
                           left_on = "shstGeometryId",
                           right_on = "id")
    
    geom_length = gpd.GeoDataFrame(all_link_gdf[['geometry']])
    geom_length.crs = all_node.crs
    geom_length = geom_length.to_crs(epsg = 26915)
    geom_length["length"] = geom_length.length

    all_link_df["length"] = geom_length["length"]

    return all_link_df, all_shape_gdf

all_link_df, all_shape_gdf = create_transit_access_link(roadway_and_rail_link_df, 
                                                        roadway_and_rail_node_gdf,
                                                        roadway_and_rail_shape_gdf)

In [450]:
all_shape_gdf.id.nunique()

41763

In [451]:
all_link_df.id.nunique()

41763

In [452]:
# number geometry increse should be the number of transit nodes: 152
print(roadway_and_rail_link_df.shstGeometryId.nunique())
print(roadway_and_rail_shape_gdf.id.nunique())
print(roadway_and_rail_shape_gdf.shape)
print(all_shape_gdf.id.nunique())
print(all_shape_gdf.shape)
print(all_link_df.shstGeometryId.nunique())

41611
41611
(41611, 6)
41763
(41763, 6)
41763


In [453]:
# number of link increase should be 2 times of transit nodes : 304

print(roadway_and_rail_link_df.shape)
print(all_link_df.shape)

(74578, 41)
(74882, 41)


In [413]:
# true shapes for line record

from shapely import ops, geometry

def get_true_line_shape(trip_df, bus_link, roadway_and_rail_shape,
                        rail_link = pd.DataFrame(columns = ['LINK_ID','shape_id', 'u', 'v'])):
    
    """
    write out true shape for each trip
    """
    
    rail_link_df = rail_link.copy()
    rail_link_df = pd.merge(trip_df[['trip_id', 'shape_id']],
                            rail_link_df,
                           how = 'right',
                           on = 'shape_id')
    
    transit_link_gdf = pd.concat([bus_link[['shape_id', 'trip_id', "shstGeometryId"]], 
                                  rail_link_df[['shape_id', 'trip_id', "shstGeometryId"]]], 
                                 sort = False, ignore_index = True)
    
    transit_link_gdf = pd.merge(transit_link_gdf,
                                roadway_and_rail_shape[['id', 'geometry']],
                                how = 'left',
                                left_on = 'shstGeometryId',
                               right_on = "id")
    
    true_line_shape_df = transit_link_gdf.groupby(['trip_id', 'shape_id'])['geometry'].agg(
                                                                lambda x: 
                                                                ops.linemerge(geometry.MultiLineString(x.tolist())))\
                                        .reset_index()
    
    """true_line_shape_df = pd.merge(true_line_shape_df, 
                                  cube,
                                 how = 'left',
                                 on = ['shape_id', 'trip_id'])"""
    
    true_line_shape_gdf = gpd.GeoDataFrame(true_line_shape_df, 
                                           crs = roadway_and_rail_shape.crs, 
                                           geometry = 'geometry')
    
    return true_line_shape_gdf

In [414]:
true_line_shape_gdf = get_true_line_shape(trip_df, 
                                                bus_link_df, 
                                                roadway_and_rail_shape_gdf,
                                                rail_link_gdf)

In [429]:
true_line_shape_gdf.to_file("../gtfs_transit/output/transit_route.geojson",
                           driver = "GeoJSON")

In [430]:
true_line_shape_gdf.columns

Index(['trip_id', 'shape_id', 'geometry'], dtype='object')

In [278]:
def link_df_to_geojson(df, properties):
    """
    Author: Geoff Boeing:
    https://geoffboeing.com/2015/10/exporting-python-data-geojson/
    """
    geojson = {"type":"FeatureCollection", "features":[]}
    for _, row in df.iterrows():
        feature = {"type":"Feature",
                   "properties":{},
                   "geometry":{"type":"LineString",
                               "coordinates":[]}}
        feature["geometry"]["coordinates"] = [[x, y] for (x,y) in list(row["geometry"].coords)]
        for prop in properties:
            feature["properties"][prop] = row[prop]
        geojson["features"].append(feature)
    return geojson


def point_df_to_geojson(df: pd.DataFrame, properties: list):
    """
    Author: Geoff Boeing:
    https://geoffboeing.com/2015/10/exporting-python-data-geojson/
    """
    
    geojson = {"type": "FeatureCollection", "features": []}
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "properties": {},
            "geometry": {"type": "Point", "coordinates": []},
        }
        feature["geometry"]["coordinates"] = [row["geometry"].x, row["geometry"].y]
        for prop in properties:
            feature["properties"][prop] = row[prop]
        geojson["features"].append(feature)
    return geojson

def fill_na(df_na):
    """
    fill str NaN with ""
    fill numeric NaN with 0
    """
    df = df_na.copy()
    num_col = list(df.select_dtypes([np.number]).columns)
    print("numeric columns: ", num_col)
    object_col = list(df.select_dtypes(['object']).columns)
    print("str columns: ", object_col)
    
    for x in list(df.columns):
        if x in num_col:
            df[x].fillna(0, inplace = True)
        elif x in object_col:
            df[x].fillna("", inplace = True)
    
    return df

In [417]:
all_shape_gdf = fill_na(all_shape_gdf)

numeric columns:  []
str columns:  ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId', 'geometry']


In [454]:
int_col = ["bike_access", "walk_access", "drive_access", "transit_access", "LANES"]
for c in int_col:
    all_link_df[c] = all_link_df[c].fillna(0).astype(np.int64)
    
all_link_df = fill_na(all_link_df)

numeric columns:  ['LANES', 'bike_access', 'drive_access', 'forward', 'length', 'tomtom_f_jnctid', 'tomtom_id', 'tomtom_t_jnctid', 'walk_access', 'rail_traveltime', 'transit_access']
str columns:  ['access', 'area', 'bridge', 'est_width', 'fromIntersectionId', 'highway', 'id', 'junction', 'key', 'landuse', 'lanes', 'link', 'maxspeed', 'name', 'nodeIds', 'oneWay', 'oneway', 'ref', 'roadClass', 'roadway', 'roundabout', 'service', 'shstGeometryId', 'shstReferenceId', 'toIntersectionId', 'tunnel', 'u', 'v', 'wayId', 'width']


In [419]:
int_col = ["bike_access", "walk_access", "drive_access", "transit_access"]
for c in int_col:
    roadway_and_rail_node_gdf[c] = roadway_and_rail_node_gdf[c].fillna(0).astype(np.int64)
    
roadway_and_rail_node_gdf = fill_na(roadway_and_rail_node_gdf)

numeric columns:  ['osm_node_id', 'drive_access', 'walk_access', 'bike_access', 'transit_access']
str columns:  ['shst_node_id', 'geometry']


In [420]:
pd.crosstab(all_link_df.transit_access, all_link_df.walk_access)
pd.crosstab(roadway_and_rail_node_gdf.transit_access, roadway_and_rail_node_gdf.walk_access)

walk_access,0,1
transit_access,,
0,150,27550
1,0,152


In [436]:
all_link_df.info()
all_shape_gdf.info()
roadway_and_rail_node_gdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74882 entries, 0 to 74881
Data columns (total 41 columns):
LANES                 74882 non-null int64
access                74882 non-null object
area                  74882 non-null object
bike_access           74882 non-null int64
bridge                74882 non-null object
drive_access          74882 non-null int64
est_width             74882 non-null object
forward               74882 non-null float64
fromIntersectionId    74882 non-null object
highway               74882 non-null object
id                    74882 non-null object
junction              74882 non-null object
key                   74882 non-null object
landuse               74882 non-null object
lanes                 74882 non-null object
length                74882 non-null float64
link                  74882 non-null object
maxspeed              74882 non-null object
name                  74882 non-null object
nodeIds               74882 non-null object
oneWay      

In [422]:
%%time

print("-------write out link shape geojson---------")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(all_shape_gdf, shape_prop)

with open("../tests/networkstandard/step2_transit/sf_shape.geojson", "w") as f:
    json.dump(shape_geojson, f)

-------write out link shape geojson---------
Wall time: 9.14 s


In [455]:
%%time

# write out link variable json
# link unique handle "shstReferenceId" + "shstGeometryId"

print("-------write out link json---------")

link_prop = all_link_df.drop(["forward", "roadClass", "oneway"], axis = 1).columns.tolist()

out = all_link_df[link_prop].to_json(orient = "records")

with open('../tests/networkstandard/step2_transit/sf_link.json', 'w') as f:
    f.write(out)

-------write out link json---------
Wall time: 4.35 s


In [424]:
%%time

print("-------write out node geojson---------")

node_prop = roadway_and_rail_node_gdf.drop("geometry", axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(roadway_and_rail_node_gdf, node_prop)

with open("../tests/networkstandard/step2_transit/sf_node.geojson", "w") as f:
    json.dump(node_geojson, f)

-------write out node geojson---------
Wall time: 5.84 s
